# Process stackoverflow data

In [1]:
%load_ext autoreload
%autoreload 2

from google.cloud import bigquery
import ast
import glob
import re
from pathlib import Path

import astor
import pandas as pd
import spacy
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split

from general_utils import apply_parallel, flattenlist
from bs4 import BeautifulSoup
from textblob import TextBlob

EN = spacy.load('en')
pd.set_option('display.max_colwidth', -1)
tqdm.pandas(desc="Processing:")

In [2]:
use_cache = False

In [3]:
# TODO: use csv for now, fix credential issue
# bigquery
# https://console.cloud.google.com/bigquery?sq=784649123063:df10117d54ab4ef2a853ce5ba8e583dc

# if use_cache == False: 
#     # get data from bigquery
#     client = bigquery.Client()
#     sql = """
#     SELECT id, body, last_activity_date, score, text as comment
#     FROM `bigquery-public-data.stackoverflow.posts_answers` as answer
#     JOIN (SELECT post_id, text from `bigquery-public-data.stackoverflow.comments`) as c on c.post_id = answer.id
#     WHERE 
#     ( Date(last_activity_date) >= Date("2015-01-01")
#       AND 
#       answer.parent_id IN (
#         SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions`
#         WHERE LOWER(tags) LIKE '%python%'
#       ))

#     """
#     df = client.query(sql).to_dataframe()
#     df.to_csv("data/processed_data/stackoverflow/raw.csv")

# Read from csv files

In [4]:
%%time
# read from csv
df = pd.concat([pd.read_csv(f"data/stackoverflow/processed_data/raw/raw-{i}.csv") for i in range(4)])

CPU times: user 22.5 s, sys: 3.45 s, total: 26 s
Wall time: 26 s


In [5]:
df.head()

,id,body,last_activity_date,score,comment
0,33734452,"<p>You can call <code>apply</code> and convert your dict values into a set can convert the <code>intersection</code> to a list:</p>\n\n<pre><code>In [15]:\nd={'a': ['j','c'],\n 'b': ['p','r','q'],\n 'c': ['n','k','l','x','a']}\nd\n\nOut[15]:\n{'a': ['j', 'c'], 'b': ['p', 'r', 'q'], 'c': ['n', 'k', 'l', 'x', 'a']}\n\nIn [17]:\ndf['points'] = df.apply(lambda row: list(set(d[row['id']]).intersection(row['points'])), axis=1)\ndf\n\nOut[17]:\n id points\n0 a [c]\n1 b []\n2 c [a, x]\n</code></pre>\n\n<p>As to why you get an error, you're trying to call a <code>.str</code> method on a Series that is a dtype list, they are not strings.</p>",2015-11-16 11:37:21.32 UTC,1,I am getting a key error. Is there any other solution using the two dataframes i added to my post?
1,5099990,"<blockquote>\n <p>One thing I've found really frustrating is that when you trap an import error you're really going down a deep rabbit hole right away.</p>\n</blockquote>\n\n<p>Take the hint.</p>\n\n<p>Do <strong>not</strong> trap import errors ever. The <code>manage.py</code> example is not a best practice.</p>\n\n<p>Do not do that.</p>\n\n<p>import errors should be very, very rare.</p>",2011-02-24 03:15:36.757 UTC,-1,"@S.Lott - I don't mean me as the developer ""know"", I mean in the control I want my code to be able to ""know"" what import failed. Again, defer to Django. That helpful message could not be given, if you relied on Python's built-in exc handling."
2,22650931,"<p>In your code sample, above <code>for rownum in range(1, r+1):</code> place <code>buff = []</code> so that:</p>\n\n<pre><code>buff = []\nfor rownum in range(1, r+1):\n</code></pre>\n\n<p>Then, where you have <code>print(list_v)</code> do the following:</p>\n\n<pre><code>buff.append(list_v)\n</code></pre>\n\n<p>Then your code sample will look like</p>\n\n<pre><code>def make_x_table_right (r,c):\n buff = []\n for rownum in range(1, r+1):\n list_v = []\n for colnum in range(1, c+1):\n v = colnum*rownum\n list_v.append(v)\n buff.append(list_v)\n return buff\n</code></pre>\n\n<p>Make sense? </p>\n\n<p>Also, since your multiplication table is probably going to be square, I'd suggest just passing one parameter <code>n</code> and using <code>n</code> in place of <code>r</code> and <code>c</code>. One other thing, the list comprehension that another user posted as a one-liner comment is perfectly fine, too. Also, it's convention to use variables <code>i</code> and <code>j</code> for matrix loop identifiers. so instead of <code>rownum</code> you'd say <code>i</code> and <code>colnum</code> you'd say <code>j</code>.</p>",2014-03-26 03:28:20.523 UTC,0,"Yup, I thought his shorter method was really cool. But is it possible to actually return the list of lists in a column format? I can generate the horizontal list of lists easy....."
3,49524124,"<p>Only one Change is need. And you can use django-import-export </p>\n\n<p><strong>models.py</strong></p>\n\n<pre><code> class Compositions(models.Model):\n composer_key = models.ForeignKey(\n Composer,\n )\n composition = models.CharField(\n max_length=383,\n unique=False\n )\n date_created = models.DateTimeField(default=timezone.now)\n\n class Meta(object):\n unique_together = (('composer_key','composition'),)\n</code></pre>\n\n<p>override save_instance with try. And ignore error when fail. \n<strong>admin.py</strong></p>\n\n<pre><code> class CompositionsResource(resources.ModelResource):\n\n class Meta:\n model = Compositions\n skip_unchanged = True\n report_skipped = True\n\n def save_instance(self, instance, using_transactions=True, dry_run=False):\n try:\n super(CompositionsResource, self).save_instance(instance, using_transactions, dry_run)\n except IntegrityError:\n pass\n\n class CompositionsAdmin(ImportExportModelAdmin):\n resource_class = CompositionsResource\n\n admin.site.register(Compositions, CompositionsAdmin)\n</code></pre>\n\n<p>and import this</p>\n\n<pre><code>from django.db import Integri

In [6]:
df.shape

(1522068, 5)

In [7]:
def tokenize_text(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]


def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)


# Aggregate comments

In [8]:
# concat comments for each post

grouped = df.groupby(['id', 'body','last_activity_date','score'],as_index=False).comment.agg(lambda x: "\n".join(x))
deduped_df = pd.DataFrame(grouped)

In [9]:
deduped_df.head(1)

,id,body,last_activity_date,score,comment
0,538,"<p>One possibility is Hudson. It's written in Java, but there's integration with Python projects:</p>\n\n<blockquote>\n <p><a href=""http://redsolo.blogspot.com/2007/11/hudson-embraces-python.html"" rel=""noreferrer"">Hudson embraces Python</a></p>\n</blockquote>\n\n<p>I've never tried it myself, however.</p>\n\n<p>(<strong>Update</strong>, Sept. 2011: After a trademark dispute Hudson has been renamed to <a href=""http://jenkins-ci.org/"" rel=""noreferrer"">Jenkins</a>.)</p>",2013-01-28 03:54:17.217 UTC,28,"@dbr: Not totally true. The language something is written in often influences the software it works with best... build systems especially. When I looked at Hudson ~2 years ago it wanted unit test output only in JUnit's XML format; my Python unit test suite didn't do that. But if my project were in Java, it would have worked out of the box. In any case, it looks like integration has come a long way in both Hudson and Python's tools since then, and I am thankful for that.\nHudson being written in Java is as relevant as your the editor you use being written in C++! It's just a very well written, easy to setup and generally fully featured CI tool. Python can be very well integrated, `nosetests --with-xunit --with-coverage`, pylint with the ""Violations"" plugins, and ""Cobertura"" for coverage\nyep jenkins/hudson is perfectly fine, a breeze to set up."


In [10]:
deduped_df.shape

(683233, 5)

# Extract text from post body

In [31]:
%%time

content_list = []
url_list = []
comment_list = []
sentiment_polarity_list = []
sentiment_subjectivity_list = []
vote_list =[]

for i, row in deduped_df.iterrows():
    content = row.body
    soup = BeautifulSoup(content, 'lxml')
    if soup.code: soup.code.decompose()
    tag_p = soup.p
    tag_pre = soup.pre
    text = ''
    if tag_p: text = text + tag_p.get_text()
    if tag_pre: text = text + tag_pre.get_text()
    if text =='': 
        continue
    tokens = tokenize_text(text)
    if len(tokens) <=3:
        continue
        
    content_list.append(' '.join(tokens))
    url_list.append('https://stackoverflow.com/a/' + str(row.id))
    comment_list.append(row.comment)
    vote_list.append(row.score)
    
    sentiment = TextBlob(row.comment).sentiment
    sentiment_polarity_list.append(sentiment.polarity)
    sentiment_subjectivity_list.append(sentiment.subjectivity)


content_token_df = pd.DataFrame({'content_token': content_list, 'url': url_list, 'vote':vote_list,'comment': comment_list, 'sentiment_polarity': sentiment_polarity_list, 'sentiment_subjectivity':sentiment_subjectivity_list})

CPU times: user 40min 10s, sys: 566 ms, total: 40min 10s
Wall time: 40min 10s


In [32]:
content_token_df.shape

(644466, 6)

# Partition train/test/validate data

In [33]:
# train, valid, test splits
s = content_token_df.url.value_counts()
# print(len(s[s != 1].index))
print(content_token_df[pd.isna(content_token_df["url"])])

# print(content_token_df.groupby("url").filter(lambda x: len(x) > 1))

# total = list(content_token_df.groupby('url'))
# print("total number of row: ", str(len(total)))


Empty DataFrame
Columns: [content_token, url, vote, comment, sentiment_polarity, sentiment_subjectivity]
Index: []


In [34]:
train, test = train_test_split(list(content_token_df.groupby('url')), train_size=0.87, shuffle=True, random_state=8081)
train, valid = train_test_split(train, train_size=0.82, random_state=8081)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [35]:
train = pd.concat([d for _, d in train]).reset_index(drop=True)
test = pd.concat([d for _, d in test]).reset_index(drop=True)
valid = pd.concat([d for _, d in valid]).reset_index(drop=True)

In [36]:
train.head()

,content_token,url,vote,comment,sentiment_polarity,sentiment_subjectivity
0,this is not a valid json string . use double quotes :,https://stackoverflow.com/a/6462436,2,"And my hours of being nearly driven insane come down to double quotes. Thanks, that fixed it.\nRequired reading: http://json.org/",-0.125926,0.348148
1,"this is not a regular sentence where words are joined with an underscore . since you are just checking if the word is present , you may either remove ( as it is matching on a word boundary and _ is a word character ! ) or add alternatives : import re my_other_string = ' the_boat_has_sunk ' my_list = [ ' car ' , ' boat ' , ' truck ' ] my_list = re.compile(r'(?:\b|_)(?:%s)(?=\b|_ ) ' % ' |'.join(my_list ) ) if re.search(my_list , my_other_string ) : print('yay ' )",https://stackoverflow.com/a/30383962,3,"I should've thought about it more clearly, because it musn't match if for example boathouse is in the string. So I need to remove all special characters. Am I correct that I have to split these with a | ? If so how do I remove a '|'?\nWorks like a charm! Thanks a lot!",0.324107,0.413690
2,"lists are zero indexed , so you need to set",https://stackoverflow.com/a/37092245,0,Oh of course. Thanks a lot! Should work fine. :),0.388889,0.566667
3,"in this case , a simple solution is to use dictionary comprehension :",https://stackoverflow.com/a/47307980,0,"quite elegant, but doesn't solve the two questions - word boundaries and stemming of n-grams",0.500000,1.000000
4,"it is likely your input file exceeds the soft memory limit in size . for big files use either or blobstoreziplineinputreader.def time_count_map(data ) : "" "" "" time count map function . "" "" "" text = data[1 ] try : reader = csv.reader([text.replace('\0 ' , ' ' ) ] , skipinitialspace = true ) for s in reader : "" "" "" calculate time elapsed "" "" "" sdw = s[1 ] start_date = time.strptime(sdw,""%m/%d/%y % i:%m:%s%p "" ) edw = s[2 ] end_date = time.strptime(edw,""%m/%d/%y % i:%m:%s%p "" ) time_difference = time.mktime(end_date ) - time.mktime(start_date ) yield ( s[0 ] , time_difference ) except indexerror , e : logging.debug(e )",https://stackoverflow.com/a/9256062,2,This worked very well for me! Thanks a lot! :),0.333333,0.500000


In [24]:
test.head()

,content_token,url,comment,sentiment_polarity,sentiment_subjectivity
0,"first , create a list containing one hundred different colors . each color should be an rgb tuple , ranging from zero to one . there are many ways to do this , but i just typed in my favorite colors by hand .",https://stackoverflow.com/a/18622422,"Hi Kevin, I tried the latter code and it worked great. Plus I get the idea now of how this loop thing actually works (or at least I am getting there) I don't know why I had trouble figuring out the loop concept with the design, but not with the colors...Anyways I really do appreciate the help.I actually had to drop this class last semester, and I really need to pass it this time around. My specialty is not programming. I am a specialist at Maya and 3DS MAX modeling, texturing, and animation. That stuff I know, this stuff I have trouble with:(...Thanks again Kevin:)",0.191667,0.35
1,"i 've had a similar problem and i 've ended up adding a counter into the album equivalent . if the count is 0 and the operation is delete ( ) , then the album object is delete()d .",https://stackoverflow.com/a/5185748,IMO Fabio's solution is quite to the point :),0.191667,0.35
2,two options that do n't require copying the whole set :,https://stackoverflow.com/a/59841,"`next(iter(your_list or []), None)` to handle None sets and empty sets\n@highBandWidth: I think python will raise an error if you do that. You're not allowed to iterate through a set that you're removing elements from.\nnext(iter(s)) is also OK and I tend to think it reads better. Also, you can use a sentinel to handle the case when s is empty. E.g. next(iter(s), set()).\nThis answers my question. Alas, I guess I will still use pop(), since iteration seems to sort the elements. I would prefer them in random order...\n+1 `iter(s).next()` is not gross but great. Completely general to take arbitrary element from any iterable object. Your choice if you want to be careful if the collection is empty though.",0.191667,0.35
3,"i do n't think you can do this in one database query using django orm . but if it does n't bothers you , i would create a custom method on a model :",https://stackoverflow.com/a/2659296,"Thanks. Note that in the example you gave, the order was ascending (in other words ""the lower the score, the higher the ranking""), so I went with it. If higher score should actually increase the ranking you should change ""score__lt"" to ""score__gt"".\nThank you, that's exactly what I was looking for.\nThis should be computationally expensive, as it will run the ranking aggregate query for each row in `Model.objects.all()`\n@mehmet Yes, of course. That's what I was trying to say in the first paragraph of this post. Back then (6 years ago) there wasn't a more effective way to do this using Django ORM. Nowadays there might be a PostgreSQL only solution using PostgreSQL window functions and a custom annotate function. I might look into that when I have more time.",0.191667,0.35
4,for python 3 :,https://stackoverflow.com/a/26097790,"@MartijnPieters Do you know what version introduced that?\nActually, the double exec within the `def exec_` is unnecessary: `exec (code) in globs, locs` with extra parenthesis around `code` does not result in `SyntaxError` in Python 3",0.191667,0.35


In [37]:
valid.head()

,content_token,url,vote,comment,sentiment_polarity,sentiment_subjectivity
0,"if you read the docs you 'll see that the comment has an attribute ( unless it was deleted ) , so you should be able to do : response_text = ' good morning to you too , { } ! ' ... c.reply(response_text.format(c.author ) )",https://stackoverflow.com/a/39419165,0,Just add /u/ to the format string. But I've always had notifications when my comments were replied to even without mentions,0.000000,0.000000
1,"i think it would be better to use "" cls "" :",https://stackoverflow.com/a/28626798,3,"the OP is also asking why `self` is not something like `inst`\nI think difference between cls and self is quite clear: cls means class, and self means instance, it's common approach to naming in python.\nI am not asking the OP is",-0.100000,0.441667
2,the best python gui toolkit is wxpython ( also known as wxwidgets ) .,https://stackoverflow.com/a/4252766,1,It's not that hard to add your own renderer.,0.154167,0.770833
3,"of course you could pass a list , but also you could just use * args . it sometimes depends on how you expect to use the function .",https://stackoverflow.com/a/32126746,7,"Okay the *args is new to me. But it's kinda the same, just different brackets. I get it now.\nNice Pythonic way\nThere's a small error in the code: the return statements are inside the for loop.\nThere is also **kwargs, you should really get to know them as they are integral to the language. http://stackoverflow.com/questions/3394835/args-and-kwargs",0.169481,0.468506
4,yes and yes . you can get rid of the loops and it will speed up things :,https://stackoverflow.com/a/49798710,2,"This is awesome, Thanks Paul! The speed up is immense.",0.416667,0.733333


In [38]:
assert valid.shape[0] + test.shape[0] + train.shape[0] == content_token_df.shape[0],f'Row count mismatch.'

# Write to file

In [39]:
def write_to(df, filename, path='./data/stackoverflow/processed_data'):
    "Helper function to write processed files to disk."
    out = Path(path)
    out.mkdir(exist_ok=True)
    df.content_token.to_csv(out/'{}.content_token'.format(filename), index=False)
    df.url.to_csv(out/'{}.url'.format(filename), index=False)
    df.to_csv(out/'{}.comment'.format(filename), columns=['comment', 'sentiment_polarity', 'sentiment_subjectivity'], index=False)
    df.vote.to_csv(out/'{}.vote'.format(filename), index=False)

In [40]:
write_to(train, 'train')
write_to(valid, 'valid')
write_to(test, 'test')

In [41]:
!ls -lah ./data/stackoverflow/processed_data

total 401M
drwxr-xr-x 4 root root 4.0K Dec  1 11:40 .
drwxr-xr-x 8 root root 4.0K Dec  1 09:24 ..
drwxr-xr-x 2 root root 4.0K Nov 25 10:14 .ipynb_checkpoints
drwxr-xr-x 2 root root 4.0K Nov 25 10:49 raw
-rw-r--r-- 1 root root  11M Nov 20 08:27 raw.csv
-rw-r--r-- 1 root root  11M Nov 20 08:25 raw9756.csv
-rw-r--r-- 1 root root  29M Dec  4 03:03 test.comment
-rw-r--r-- 1 root root  18M Dec  4 03:03 test.content_token
-rw-r--r-- 1 root root 3.0M Dec  4 03:03 test.url
-rw-r--r-- 1 root root 172K Dec  4 03:03 test.vote
-rw-r--r-- 1 root root 158M Dec  4 03:03 train.comment
-rw-r--r-- 1 root root  97M Dec  4 03:03 train.content_token
-rw-r--r-- 1 root root  17M Dec  4 03:03 train.url
-rw-r--r-- 1 root root 942K Dec  4 03:03 train.vote
-rw-r--r-- 1 root root  35M Dec  4 03:03 valid.comment
-rw-r--r-- 1 root root  21M Dec  4 03:03 valid.content_token
-rw-r--r-- 1 root root 3.6M Dec  4 03:03 valid.url
-rw-r--r-- 1 root root 207K Dec  4 03:03 valid.vote
